In [6]:
from selenium import webdriver
import pandas as pd
import time

In [2]:
def get_k_hadv(home_goals, away_goals, home_team_rating, away_team_rating, home_team_rating_change):
    list_k = [20, 30, 40, 50, 60]
    
    home_result = 1.0 if home_goals > away_goals else 0.5 if  home_goals == away_goals else 0.0
    goal_difference = abs(home_goals - away_goals)
    if goal_difference < 2:
        k_modifier = 1
    elif goal_difference == 2:
        k_modifier = 1.5
    elif goal_difference == 3:
        k_modifier = 1.75
    else:
        k_modifier = 7/4 + (goal_difference - 3)/8
    
    for ki in list_k:
        expected_home_result_nohadv = 1 / (1 + 10**((-home_team_rating + away_team_rating) / 400))
        htrc_nohadv = round((home_result - expected_home_result_nohadv) * ki * k_modifier)
        
        expected_home_result_hadv = 1 / (1 + 10**((-home_team_rating + away_team_rating - 100) / 400))
        htrc_hadv = round((home_result - expected_home_result_hadv) * ki * k_modifier)
        
        if home_team_rating_change == htrc_nohadv:
            k = ki
            hadv = 0
            break
        elif home_team_rating_change == htrc_hadv:
            k = ki
            hadv = 100
            break
        elif ki == 60:
            k = -1
            hadv = -1
            
    return k, hadv        

In [23]:
def get_df_row_list(row_list):
    beg = time.time()
    t0 = 0
    
    df_row_list = []
    for i, row in enumerate(row_list):
        t = time.time() - beg
        if t - t0 > 10:
            t0 = t
            print("Time {:.2f}, row {}".format(t, i), flush=True)
        
        date = row.find_element_by_xpath('.//div[1]').text.replace("\n", " ")
        tournament = row.find_element_by_xpath('.//div[4]').text.replace("\n", " ")
        teams = row.find_element_by_xpath('.//div[2]').text
        home_team = teams.split("\n")[0]
        away_team = teams.split("\n")[1]
        rating_changes = row.find_element_by_xpath('.//div[5]').text.replace("−", "-")
        home_team_rating_change = int(rating_changes.split("\n")[0])
        away_team_rating_change = int(rating_changes.split("\n")[1])
        ratings = row.find_element_by_xpath('.//div[6]').text
        home_team_new_rating = int(ratings.split("\n")[0])
        away_team_new_rating = int(ratings.split("\n")[1])
        home_team_rating = home_team_new_rating - home_team_rating_change
        away_team_rating = away_team_new_rating - away_team_rating_change
        rank_changes = row.find_element_by_xpath('.//div[7]').text.replace("−", "-")
        try:
            home_team_rank_change = int(rank_changes.split("\n")[0])
            away_team_rank_change = int(rank_changes.split("\n")[1])
        except:
            home_team_rank_change = 0
            away_team_rank_change = 0
        ranks = row.find_element_by_xpath('.//div[8]').text
        home_team_new_rank = int(ranks.split("\n")[0])
        away_team_new_rank = int(ranks.split("\n")[1])
        home_team_rank = home_team_new_rank - home_team_rank_change
        away_team_rank = away_team_new_rank - away_team_rank_change
        result = row.find_element_by_xpath('.//div[3]').text
        home_goals = int(result.split("\n")[0])
        away_goals = int(result.split("\n")[1])

        (k, hadv) = get_k_hadv(home_goals, away_goals, home_team_rating, away_team_rating, home_team_rating_change)
        rdiff = home_team_rating - away_team_rating + hadv
        
        df_row = [date, tournament, home_team, away_team, home_team_rating, away_team_rating, home_goals, away_goals, 
                  home_team_rating_change, away_team_rating_change, home_team_new_rating, away_team_new_rating, 
                  home_team_rank, away_team_rank, home_team_rank_change, away_team_rank_change, 
                  home_team_new_rank, away_team_new_rank, k, hadv, rdiff]
        df_row_list.append(df_row)
    return df_row_list

In [4]:
columns = ['date', 'tournament', 'home_team', 'away_team', 'home_team_rating', 'away_team_rating', 'home_goals', 'away_goals', 
           'home_team_rating_change', 'away_team_rating_change', 'home_team_new_rating', 'away_team_new_rating', 
           'home_team_rank', 'away_team_rank', 'home_team_rank_change', 'away_team_rank_change', 
           'home_team_new_rank', 'away_team_new_rank', 'k', 'hadv','rdiff']

In [25]:
list_years = list(reversed(range(1990,2010)))
for year in list_years:
    url = "https://www.eloratings.net/"+str(year)+"_results"
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(url)
    time.sleep(5)
    print("----- Year {} ----".format(year), flush=True)
    row_list = driver.find_elements_by_xpath('//*[@id="maintable_'+str(year)+'_results"]/div[5]/div/div')
    df_row_list = get_df_row_list(row_list)
    driver.close()
    df = pd.DataFrame(df_row_list, columns=columns)
    df.to_csv('games_'+str(year)+'.csv')

----- Year 2009 ----
Time 10.03, row 49
Time 20.19, row 101
Time 30.49, row 150
Time 40.51, row 195
Time 50.55, row 235
Time 60.58, row 271
Time 70.72, row 304
Time 80.80, row 337
Time 90.89, row 382
Time 101.10, row 427
Time 111.17, row 470
Time 121.41, row 513
Time 131.61, row 555
Time 141.75, row 596
Time 151.87, row 636
Time 162.11, row 676
Time 172.11, row 714
Time 182.23, row 752
Time 192.25, row 789
Time 202.43, row 826
Time 212.49, row 862
Time 222.70, row 898
----- Year 2008 ----
Time 10.13, row 52
Time 20.20, row 103
Time 30.24, row 153
Time 40.29, row 202
Time 50.35, row 249
Time 60.51, row 295
Time 70.71, row 340
Time 80.93, row 383
Time 91.19, row 422
Time 101.27, row 459
Time 111.28, row 496
Time 121.43, row 532
Time 131.48, row 567
Time 141.50, row 602
Time 151.72, row 637
Time 161.83, row 671
Time 171.86, row 704
Time 182.02, row 737
Time 192.33, row 770
Time 202.47, row 802
Time 212.75, row 829
Time 223.00, row 860
Time 233.11, row 890
Time 243.43, row 920
Time 253.61,

Time 131.40, row 565
Time 141.48, row 602
Time 151.71, row 639
Time 161.72, row 674
Time 171.75, row 709
Time 181.98, row 744
Time 192.03, row 778
----- Year 1992 ----
Time 10.13, row 53
Time 20.15, row 104
Time 30.31, row 154
Time 40.50, row 203
Time 50.58, row 250
Time 60.77, row 296
Time 70.97, row 341
Time 81.05, row 385
Time 91.25, row 423
Time 101.46, row 462
Time 111.69, row 497
Time 121.80, row 531
Time 132.71, row 549
Time 142.95, row 560
Time 153.18, row 602
----- Year 1991 ----
Time 10.11, row 57
Time 20.26, row 115
Time 30.32, row 169
Time 40.49, row 221
Time 50.72, row 271
Time 60.85, row 316
Time 70.91, row 356
Time 81.14, row 399
Time 91.17, row 440
----- Year 1990 ----
Time 10.00, row 53
Time 20.18, row 106
Time 30.31, row 160
Time 40.51, row 214
Time 50.65, row 265
Time 60.79, row 315
Time 70.80, row 362
Time 80.87, row 410
Time 90.99, row 457
Time 101.18, row 503


In [22]:
for i in range(694,714):
    row = row_list[i]
    teams = row.find_element_by_xpath('.//div[2]').text
    home_team = teams.split("\n")[0]
    away_team = teams.split("\n")[1]
    rank_changes = row.find_element_by_xpath('.//div[7]').text.replace("−", "-")
    print(repr(rank_changes), home_team, away_team)
    #home_team_rank_change = int(rank_changes.split("\n")[0])
    #away_team_rank_change = int(rank_changes.split("\n")[1])

'0\n-1' Cambodia Brunei
'-3\n+3' Laos Myanmar
'0\n0' Bangladesh Sri Lanka
'0\n0' Bangladesh Sri Lanka
'-\n-' Chagos Islands Somaliland
'+5\n-5' Kuwait North Korea
'+3\n-1' Philippines Nepal
'0\n+2' Bahrain North Korea
'-5\n+4' Kuwait Yemen
'+2\n-3' Qatar North Korea
'+3\n-1' Saudi Arabia Palestine
'+1\n0' United Arab Emirates Lebanon
'0\n-1' Bahrain Singapore
'0\n0' Oman Yemen
'+2\n-3' Thailand Philippines
'0\n-4' Uganda Ethiopia
'-6\n+3' Vietnam Palestine
'-1\n+2' Cuba French Guiana
'0\n0' Indonesia East Timor
'+1\n0' Trinidad and Tobago Curaçao
